In [69]:
from dotenv import load_dotenv
load_dotenv()

True

In [70]:
#Connecting to SQL Database 'Chinook.db' 

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")

print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [71]:
#Setting up Chains 
# 1. Convert Question into SQL query
# 2. Execute SQL query
# 3. Ue result, generate answer 

#Chain #1 -> Convert Question into SQL query

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini")

from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many artist are there"})

formatted_response = response.replace('SQLQuery: ', '').replace('"','')
formatted_response
db.run(formatted_response)

'[(275,)]'

In [72]:
#Chain #2 -> Execute SQL query (No human approval for querying -> to add for the future!)

from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain import Chain

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm,db)



chain = write_query | execute_query
chain.invoke({"question": "How many artist are there"})

# I have figured it out. I must format the qeury generation -> see above cell -> I need to add that to the chain 

ImportError: cannot import name 'Chain' from 'langchain' (c:\Users\elfo\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\__init__.py)

In [67]:
#Chain #3 -> Answer original prompt

from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question and remove 'SQLQuery' from the generated syntax.
    
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(result=itemgetter("query") | execute_query) | answer_prompt | llm | StrOutputParser()
)

chain.invoke({"question": "Select 1"})

'It seems there was an error in the SQL query due to an incorrect syntax at the beginning. The correct SQL query to retrieve the album title and artist name from the database should not include the "Question" prefix. \n\nThe corrected SQL query is:\n```sql\nSELECT "Album"."Title", "Artist"."Name" FROM "Album" JOIN "Artist" ON "Album"."ArtistId" = "Artist"."ArtistId" LIMIT 1;\n```\n\nThis query retrieves the title of an album along with the name of the artist. If you run this corrected query, it will return the first album title and artist name from the database.'